# PyTorch DDP Fashion MNIST Training Example

This example demonstrates how to train a convolutional neural network (CNN) to classify images using the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and [PyTorch Distributed Data Parallel (DDP)](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html).

This notebook walks you through running that example locally, and how to easily scale PyTorch DDP across multiple nodes with Kubeflow TrainJob.

## Install the Kubeflow SDK

You need to install the Kubeflow SDK to interact with Kubeflow Trainer APIs:

In [ ]:
# TODO (astefanutti): Change to the Kubeflow SDK when it's available.
# !pip install git+https://github.com/kubeflow/trainer.git@master#subdirectory=sdk

## Install the PyTorch Dependencies

You also need to install PyTorch and Torchvision to be able to run the example locally:

In [ ]:
!pip install torch==2.5.0
!pip install torchvision==0.20.0

## Define the Training Function

The first step is to create function to train CNN model using Fashion MNIST data.

In [17]:
def train_fashion_mnist():
    import os

    import torch
    import torch.distributed as dist
    import torch.nn.functional as F
    from torch import nn
    from torch.utils.data import DataLoader, DistributedSampler
    from torchvision import datasets, transforms

    # Define the PyTorch CNN model to be trained
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5, 1)
            self.conv2 = nn.Conv2d(20, 50, 5, 1)
            self.fc1 = nn.Linear(4 * 4 * 50, 500)
            self.fc2 = nn.Linear(500, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 4 * 4 * 50)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    # Use NCCL if a GPU is available, otherwise use Gloo as communication backend.
    device, backend = ("cuda", "nccl") if torch.cuda.is_available() else ("cpu", "gloo")
    print(f"Using Device: {device}, Backend: {backend}")

    # Setup PyTorch distributed.
    local_rank = int(os.getenv("LOCAL_RANK", 0))
    dist.init_process_group(backend=backend)
    print(
        "Distributed Training for WORLD_SIZE: {}, RANK: {}, LOCAL_RANK: {}".format(
            dist.get_world_size(),
            dist.get_rank(),
            local_rank,
        )
    )

    # Create the model and load it into the device.
    device = torch.device(f"{device}:{local_rank}")
    model = nn.parallel.DistributedDataParallel(Net().to(device))
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

    
    # Download FashionMNIST dataset only on local_rank=0 process.
    if local_rank == 0:
        dataset = datasets.FashionMNIST(
            "./data",
            train=True,
            download=True,
            transform=transforms.Compose([transforms.ToTensor()]),
        )
    dist.barrier()
    dataset = datasets.FashionMNIST(
        "./data",
        train=True,
        download=False,
        transform=transforms.Compose([transforms.ToTensor()]),
    )


    # Shard the dataset accross workers.
    train_loader = DataLoader(
        dataset,
        batch_size=100,
        sampler=DistributedSampler(dataset)
    )

    # TODO(astefanutti): add parameters to the training function
    dist.barrier()
    for epoch in range(1, 3):
        model.train()

        # Iterate over mini-batches from the training set
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Copy the data to the GPU device if available
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            outputs = model(inputs)
            loss = F.nll_loss(outputs, labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0 and dist.get_rank() == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(inputs),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    # Wait for the distributed training to complete
    dist.barrier()
    if dist.get_rank() == 0:
        print("Training is finished")

    # Finally clean up PyTorch distributed
    dist.destroy_process_group()

## Dry-run the Training Locally

We are going to download Fashion MNIST Dataset and start local training.

In [4]:
import os

# Set the Torch Distributed env variables so the training function can be run locally in the Notebook.
# See https://pytorch.org/docs/stable/elastic/run.html#environment-variables
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "1234"

# Run the training function locally.
train_fashion_mnist()

Using Device: cpu, Backend: gloo
Distributed Training for WORLD_SIZE: 1, RANK: 0, LOCAL_RANK: 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26.4M/26.4M [00:02<00:00, 10.9MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 1.50MB/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.42M/4.42M [00:00<00:00, 8.57MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<00:00, 4.41MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312359
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.058874
Train Epoch: 1 [2000/60000 (3%)]	Loss: 1.059449
Train Epoch: 1 [3000/60000 (5%)]	Loss: 1.130449
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.027112
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.845158
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.747619
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.731784
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.623452
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.666745
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.597833
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.634247
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.533735
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.545662
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.637130
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.596679
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.505523
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.484452
Train Epoch: 1 [18000/60000 (3

## Scale PyTorch DDP with Kubeflow TrainJob

You can use `TrainerClient()` from the Kubeflow SDK to communicate with Kubeflow Trainer APIs and scale your training function across multiple PyTorch training nodes.

`TrainerClient()` verifies that you have required access to the Kubernetes cluster.

Kubeflow Trainer creates a `TrainJob` resource and automatically sets the appropriate environment variables to set up PyTorch in distributed environment.



In [5]:
from kubeflow.trainer import Trainer, TrainerClient

client = TrainerClient()

## List the Training Runtimes

You can get the list of available Training Runtimes to start your TrainJob. Each Training Runtime shows whether you can use it for pre-training or post-training.

Additionally, it migh show available accelerator type and number of available resources.

In [18]:
for runtime in client.list_runtimes():
    print(runtime)

Runtime(name='mpi-distributed', phase='Unknown', accelerator='Unknown', accelerator_count='Unknown')
Runtime(name='torch-distributed', phase='pre-training', accelerator='gpu-tesla-v100-16gb', accelerator_count='4')


## Run the Distributed TrainJob

Kubeflow TrainJob will train the above model on 3 PyTorch nodes.

In [19]:
job_name = client.train(
    # Use one the of the training runtimes installed on your Kubernetes cluster
    runtime_ref="torch-distributed",
    trainer=Trainer(
        func=train_fashion_mnist,
        # Set how many PyTorch nodes you want to use for distributed training.
        num_nodes=3,
        # Set the resources for each PyTorch node.
        resources_per_node={
            "cpu": 3,
            "memory": "16Gi",
            # Uncomment this to distribute the TrainJob using GPU nodes.
            # "nvidia.com/gpu": 1,
        },
    ),
)

## Check the TrainJob components

You can check the components of TrainJob that's created.

Since the TrainJob performs distributed training across 3 nodes, it generates 3 components: `trainer-node-0` .. `trainer-node-2`.

You can get the individual status for each of these components.

In [20]:
import time

def wait_for_job_running():
    for _ in range(100):
        trainjob = client.get_job(name=job_name)
        for c in trainjob.components:
            if c.name == "trainer-node-0" and c.status == "Running":
                return
        print("Wait for TrainJob running status. Sleep for 5 seconds")
        time.sleep(5)

In [21]:
# TODO (andreyvelich): Use wait_for_job_status API from TrainerClient() when it is implemented.
wait_for_job_running()

In [22]:
for c in client.get_job(name=job_name).components:
    print(f"Component: {c.name}, Status: {c.status}, Devices: {c.device} x {c.device_count}\n")

Component: trainer-node-0, Status: Running, Devices: gpu x 1

Component: trainer-node-1, Status: Running, Devices: gpu x 1

Component: trainer-node-2, Status: Running, Devices: gpu x 1



## Watch the TrainJob logs

We can use the `get_job_logs()` API to get the TrainJob logs.

Since we run training on 3 GPUs, every PyTorch node uses 60,000/3 = 20,000 images from the dataset.

In [23]:
_ = client.get_job_logs(job_name, follow=True)

[trainer-node]: Using Device: cuda, Backend: nccl
[trainer-node]: Distributed Training for WORLD_SIZE: 3, RANK: 0, LOCAL_RANK: 0
[trainer-node]: Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
[trainer-node]: Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz
100%|██████████| 26.4M/26.4M [00:02<00:00, 11.1MB/s]
[trainer-node]: Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw
[trainer-node]: Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
[trainer-node]: Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz
100%|██████████| 29.5k/29.5k [00:00<00:00, 74.8MB/s]
[trainer-node]: Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionM

## Delete the TrainJob

When TrainJob is finished, you can delete the resource.


In [ ]:
# client.delete_job(job_name)